In [5]:
import os
# breeds = os.listdir("CatBreed/images")
# breeds = sorted(breeds)
# breeds.pop(0)
# breed_map = dict()
#
# for i in range(len(breeds)):
#     breed_map[i] = breeds[i]
# breed_map

{0: 'Abyssinian',
 1: 'American Bobtail',
 2: 'American Curl',
 3: 'American Shorthair',
 4: 'American Wirehair',
 5: 'Applehead Siamese',
 6: 'Balinese',
 7: 'Bengal',
 8: 'Birman',
 9: 'Bombay',
 10: 'British Shorthair',
 11: 'Burmese',
 12: 'Burmilla',
 13: 'Calico',
 14: 'Canadian Hairless',
 15: 'Chartreux',
 16: 'Chausie',
 17: 'Chinchilla',
 18: 'Cornish Rex',
 19: 'Cymric',
 20: 'Devon Rex',
 21: 'Dilute Calico',
 22: 'Dilute Tortoiseshell',
 23: 'Domestic Long Hair',
 24: 'Domestic Medium Hair',
 25: 'Domestic Short Hair',
 26: 'Egyptian Mau',
 27: 'Exotic Shorthair',
 28: 'Extra-Toes Cat - Hemingway Polydactyl',
 29: 'Havana',
 30: 'Himalayan',
 31: 'Japanese Bobtail',
 32: 'Javanese',
 33: 'Korat',
 34: 'LaPerm',
 35: 'Maine Coon',
 36: 'Manx',
 37: 'Munchkin',
 38: 'Nebelung',
 39: 'Norwegian Forest Cat',
 40: 'Ocicat',
 41: 'Oriental Long Hair',
 42: 'Oriental Short Hair',
 43: 'Oriental Tabby',
 44: 'Persian',
 45: 'Pixiebob',
 46: 'Ragamuffin',
 47: 'Ragdoll',
 48: 'Russ

In [8]:
# import csv
# with open('CatBreedMap.csv', 'w') as f:
#     w = csv.writer(f)
#     w.writerows(breed_map.items())

In [ ]:
import numpy as np # linear algebra
import os
import torch
import torchvision
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torchvision.datasets as datasets # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms # Transformations we can perform on our dataset
import torch.nn.functional as F # All functions that don't have any parameters
from torch.utils.data import DataLoader, Dataset # Gives easier dataset managment and creates mini batches
from torchvision.datasets import ImageFolder
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F
from PIL import Image

In [ ]:
#DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # use gpu or cpu

assert torch.backends.mps.is_available(), "GPU is not available, check the directions above (or disable this assertion to use CPU)"
DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cpu") # use gpu or cpu

In [ ]:
FAST_RUN = False
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3
# DATA_PATH = "/content/drive/Shareddrives/cse 455/train_data"
DATA_PATH = "CatBreed"
BATCH_SIZE = 100
SAVE_TO = "Saved"


In [ ]:

transform = transforms.Compose([
transforms.Resize((IMAGE_WIDTH, IMAGE_HEIGHT)),
transforms.ToTensor(),
transforms.Normalize(mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225))
])

In [ ]:
data = datasets.ImageFolder(DATA_PATH, transform=transform)

In [ ]:
train_size = int(0.8 * len(data))
test_size = int(0.1 * len(data))
val_size = len(data) - train_size - test_size

In [ ]:
train_size, val_size, test_size

In [ ]:
train_set, val_set, test_set = torch.utils.data.random_split(data, [train_size, val_size, test_size])

In [ ]:
train_loader = DataLoader(
train_set,
batch_size=BATCH_SIZE,
shuffle=True
)

val_loader = DataLoader(
val_set,
batch_size=BATCH_SIZE,
shuffle=True
)

test_loader = DataLoader(
test_set,
batch_size=BATCH_SIZE,
shuffle=True
)

In [ ]:
class CatBreedCNN(nn.Module):
    def __init__(self):
        super().__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size=(3, 3), stride=2)
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size=(3, 3), stride=2)
        self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=(3, 3), stride=2)

        # conected layers
        self.fc1 = nn.Linear(in_features= 2304, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=50)
        self.fc3 = nn.Linear(in_features=50, out_features=2)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)

        # flatten
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [ ]:
model = CatBreedCNN().to(DEVICE)

In [ ]:
optimizer = optim.Adam(model.parameters(), 0.0015)

In [ ]:
def train():
    EPOCHS = 1

    loss = nn.CrossEntropyLoss()
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
    # print_every = 100

    for e in range(EPOCHS):

      model.train()
      train_loss = 0.0
      train_acc = 0.0

      # itr = 0
      # start = time.time()

      for (x_batch, labels) in train_loader:
        x_batch, labels = x_batch.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        labels_pred = model(x_batch)
        batch_loss = loss(labels_pred, labels)
        train_loss = train_loss + batch_loss.item()

        labels_pred_max = torch.argmax(labels_pred, 1)
        batch_acc = torch.sum(labels_pred_max == labels)
        train_acc = train_acc + batch_acc.item()

        batch_loss.backward()
        optimizer.step()

        train_losses.append(train_loss / len(train_loader))
        train_accuracies.append(train_acc / (BATCH_SIZE * len(train_loader)))


        # itr = itr + 1
        # if iter % print_every == 0:
        #       print_loss_avg = train_loss / print_every
        #       print('%s (%d %d%%) %.4f' % (timeSince(start, iter / len(train_loader)),
        #                                     iter, iter / len(train_loader) * 100, print_loss_avg))
        # Validation loop; use .no_grad() context manager to save memory.
        model.eval()
        val_loss = 0.0
        val_acc = 0.0




        with torch.no_grad():
          for (v_batch, labels) in val_loader:
              v_batch, labels = v_batch.to(DEVICE), labels.to(DEVICE)
              labels_pred = model(v_batch)
              v_batch_loss = loss(labels_pred, labels)
              val_loss = val_loss + v_batch_loss.item()

              v_pred_max = torch.argmax(labels_pred, 1)
              batch_acc = torch.sum(v_pred_max == labels)
              val_acc = val_acc + batch_acc.item()
        val_losses.append(val_loss / len(val_loader))
        val_accuracies.append(val_acc / (BATCH_SIZE * len(val_loader)))
      print("Epoch: {}, val loss: {:.4f}, val acc: {:.4f}, train loss: {:.4f}, train acc: {:.4f},\n".format(e, val_losses[e], val_accuracies[e], train_losses[e], train_accuracies[e]))

In [ ]:
CatBreedCNN.train()
torch.save(CatBreedCNN.state_dict(), SAVE_TO)

In [ ]:
def evaluate(model: nn.Module, loader: DataLoader) :
  loss = nn.CrossEntropyLoss()
  model.eval()
  test_loss = 0.0
  test_acc = 0.0
  with torch.no_grad():
    for (batch, labels) in loader:
      batch, labels = batch.to(DEVICE), labels.to(DEVICE)
      y_batch_pred = model(batch)
      batch_loss = loss(y_batch_pred, labels)
      test_loss = test_loss + batch_loss.item()

      pred_max = torch.argmax(y_batch_pred, 1)
      batch_acc = torch.sum(pred_max == labels)
      test_acc = test_acc + batch_acc.item()
    test_loss = test_loss / len(loader)
    test_acc = test_acc / (BATCH_SIZE * len(loader))
    return test_loss, test_acc

In [ ]:
test_loss, test_acc = evaluate(model, test_loader)
print(f"Test Accuracy: {test_acc}")